### Import and Read

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as mplt
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from IPython.core.interactiveshell import InteractiveShell
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import explained_variance_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
import matplotlib.dates as mdates
%matplotlib inline
import seaborn as sns
import math
import gc
import ipaddress
from urllib.parse import urlparse
from tldextract import extract
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

from data_science_utils import dataframe as df_utils
from data_science_utils import models as model_utils
from data_science_utils import plots as plot_utils
from data_science_utils.dataframe import column as column_utils

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from IPython.display import display, HTML


from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix,classification_report

from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import RobustScaler

import lightgbm as lgb

np.set_printoptions(threshold=np.nan)




plt.rcParams["figure.figsize"] = (24,4)

from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score
import missingno as msno
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets
from sklearn.decomposition import PCA
import datetime
from scipy import signal
import matplotlib.pyplot as plt
from datetime import timedelta
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score



from sklearn import linear_model



InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_seq_items', None)
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import RobustScaler
import warnings
warnings.filterwarnings('ignore')

import sys,os
sys.path.append(os.getcwd())

Using TensorFlow backend.
/Users/ahemf/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [27]:
df = pd.read_csv("data/application_train.csv.zip")
df_test = pd.read_csv("data/application_test.csv.zip")

df_bb = pd.read_csv("data/bureau_balance.csv.zip")
df_b = pd.read_csv("data/bureau.csv.zip")
df_pa = pd.read_csv("data/bureau_balance.csv.zip")
df_ccb = pd.read_csv("data/credit_card_balance.csv.zip")

df_ip = pd.read_csv("data/installments_payments.csv.zip")
df_pcb = pd.read_csv("data/POS_CASH_balance.csv.zip")
df_pa = pd.read_csv("data/previous_application.csv.zip")


In [28]:
def lower_case_cols(df):
    df.columns=list(map(lambda x:x.lower(),df.columns))
    
list(map(lower_case_cols,[df,df_test,df_bb,df_b,df_pa,df_ccb,df_ip,df_pcb,df_pa]))

[None, None, None, None, None, None, None, None, None]

### Feature Engg

In [29]:
cols=df_utils.get_column_datatypes(df)
object_cols = list(cols[cols['datatype']=='object'].index)
le={}
for col in object_cols:
    le[col] = column_utils.label_encode_text_column(df,col,df_test=df_test,fillna="-1")

In [30]:
cols_df=df_utils.get_column_datatypes(df)
object_cols = list(cols_df[cols_df['datatype']=='object'].index)

num_cols = list(set(cols_df.index) - set(object_cols))



features_base = list(set(num_cols) - set(['target']))

In [21]:
X = df[features_base]
y = df['target']
model.fit(X,y)
fi=model_utils.feature_importance(model,df,features)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, eval_metric='auc', gamma=16, learning_rate=0.1,
       max_delta_step=0, max_depth=6, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=4, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=40,
       reg_lambda=40, scale_pos_weight=4, seed=None, silent=True,
       subsample=1)

In [31]:
flag_docs = df_utils.get_specific_cols(df,prefix='flag_document_')

df['flag_document_sum'] = df[flag_docs].sum(axis=1)
df_test['flag_document_sum'] = df_test[flag_docs].sum(axis=1)

In [6]:
#xgc_gpu = XGBClassifier(max_depth=12, learning_rate=0.1,n_estimators=300,missing=np.nan,gamma=8,eval_metric="auc",reg_lambda=0.5,reg_alpha=0.5,tree_method='gpu_exact')

In [34]:
def conf_mat(model,X,y):
    y_scores = model.predict(X)
    return model_utils.confusion_matrix_frame(y,y_scores)

### Experiments

#### Experiment 1: Base features

In [35]:
model = XGBClassifier(max_depth=6, learning_rate=0.1,n_estimators=100,n_jobs=4,scale_pos_weight=4,missing=np.nan,gamma=16,eval_metric="auc",reg_lambda=40,reg_alpha=40)
baseline = model_utils.cross_validate_classifier(model,X,y,cv=4)
baseline['test_roc_auc']
baseline['test_roc_auc_mean']
baseline['train_roc_auc']
baseline['train_roc_auc_mean']

array([ 0.75760226,  0.75774505,  0.75557272,  0.75740295])

0.75708074484408883

array([ 0.78137358,  0.78217083,  0.78278008,  0.78089733])

0.78180545734391649

#### Experiment 2: Remove features with zero importance

In [36]:
features_v1=list(fi[fi["importance"]>0]['feature'].values)

In [37]:
X = df[features_v1]
y = df['target']
exp2 = model_utils.cross_validate_classifier(model,X,y,cv=4)
exp2['test_roc_auc']
exp2['test_roc_auc_mean']
exp2['train_roc_auc']
exp2['train_roc_auc_mean']


array([ 0.75820862,  0.75779092,  0.75606301,  0.75754596])

0.75740212587325972

array([ 0.78233534,  0.78275799,  0.78285542,  0.78169157])

0.78241007948624519

#### Experiment 3: 'flag_document_sum' with base features

In [38]:
features_v2 = list(features_base)
features_v2.append('flag_document_sum')

In [39]:
X = df[features_v2]
y = df['target']
exp3 = model_utils.cross_validate_classifier(model,X,y,cv=4)
exp3['test_roc_auc']
exp3['test_roc_auc_mean']
exp3['train_roc_auc']
exp3['train_roc_auc_mean']

array([ 0.75832338,  0.75732562,  0.75622024,  0.75722937])

0.75727465324556076

array([ 0.78283138,  0.78259719,  0.78259774,  0.78113628])

0.78229064835196005

#### Experiment 4: 'flag_document_sum' with Exp 2 features, zero importance features removed

In [40]:
features_v3 = list(features_v1)
features_v3.append('flag_document_sum')

In [41]:
X = df[features_v3]
y = df['target']
exp4 = model_utils.cross_validate_classifier(model,X,y,cv=4)
exp4['test_roc_auc']
exp4['test_roc_auc_mean']
exp4['train_roc_auc']
exp4['train_roc_auc_mean']

ValueError: feature_names must be unique

### Submission

In [26]:
model_utils.generate_results(model,df_test,features,"sk_id_curr","target","results/results-4.csv")

In [27]:
%pwd
%cd  results

!kaggle  competitions  submit -c home-credit-default-risk -f results-4.csv -m "Xgb base"
%cd ..

'/home/ubuntu/data-science/kaggle/home-credit-default-risk'

/home/ubuntu/data-science/kaggle/home-credit-default-risk/results


['Successfully submitted to Home Credit Default Risk']

/home/ubuntu/data-science/kaggle/home-credit-default-risk


In [28]:
!kaggle competitions submissions -c home-credit-default-risk -q

fileName       date                 description  status    publicScore  privateScore  
-------------  -------------------  -----------  --------  -----------  ------------  
results-4.csv  2018-05-27 06:39:32  Xgb base     complete  0.741        None          
results-4.csv  2018-05-27 06:22:13  Xgb base     complete  0.743        None          
results-3.csv  2018-05-26 20:10:22  Xgb base     complete  0.742        None          
results-2.csv  2018-05-26 20:09:25  Xgb base     complete  0.743        None          
results.csv    2018-05-26 19:50:53  Xgb base     complete  0.744        None          
